In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException
import time
import mysql.connector

data_list = [] 
# 웹 드라이버 설정 (예: Chrome)
driver = webdriver.Chrome()

# 1. 페이지 로드
url = "https://www.kebhana.com/cms/rate/index.do?contentUrl=/cms/rate/wpfxd651_07i.do#//HanaBank"
driver.get(url)

# 2. 페이지 로드 대기
wait = WebDriverWait(driver, 10)

state = "CNY"
# 3. 로딩 레이어가 사라질 때까지 대기
try:
    loading_layer = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#OPB_loadingLayerID_generatedByJSOPB_modalMaskID_generatedByJS")))
    wait.until(EC.invisibility_of_element(loading_layer))
except TimeoutException:
    print("Loading layer did not disappear, continuing...")

# 4. "기간환율변동" 선택 클릭
try:
    period_exchange_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "label[for='inqType_p'].radioForm")))
    period_exchange_button.click()
except (TimeoutException, ElementClickInterceptedException):
    driver.execute_script("document.querySelector(\"label[for='inqType_p']\").click();")

# 5. 날짜 수정 (조회 기간 시작일)
date_input = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#tmpInqStrDt_p")))
date_input.clear()
date_input.send_keys("2023-08-29")

# 6. 날짜 수정 (조회 기간 종료일)
date_input = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#tmpInqEndDt_p")))
date_input.clear()
date_input.send_keys("2024-09-13")

# 7. 통화 선택 (예: USD)
currency_select = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#curCd")))
select = Select(currency_select)
select.select_by_value(state)

# 8. "최종" 라디오 버튼 선택
try:
    final_radio_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "label[for='tmpPbldDvCd_2']")))
    final_radio_button.click()
except (TimeoutException, ElementClickInterceptedException):
    driver.execute_script("document.querySelector(\"label[for='tmpPbldDvCd_2']\").click();")

# 9. "조회" 버튼 클릭
query_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#HANA_CONTENTS_DIV > div.btnBoxCenter > a")))
query_button.click()

# 10. 조회 후 5초 대기
time.sleep(5)

# 11. 테이블 데이터 크롤링
rows = driver.find_elements(By.CSS_SELECTOR, "#searchContentDiv > div.printdiv > table > tbody > tr")
for row in rows:
    columns = row.find_elements(By.TAG_NAME, "td")
    data = [column.text for column in columns]
    
    # 첫 번째 값 제외, 나머지를 float로 변환
    for i in range(1, len(data)):
        try:
            data[i] = float(data[i].replace(',', ''))  # 쉼표 제거 후 float 변환
        except ValueError:
            print(f"변환 실패: {data[i]}")
    data_list.append(data)
    print("Row:", data)

# 웹 드라이버 종료
driver.quit()

# MySQL 연결 정보 설정
config = {
    'user': 'yangjinan',
    'password': 'ALsk85200!!',
    'host': 'hanaex.mysql.database.azure.com',
    'database': 'my_database',
    'port': 3306
}

# MySQL 연결
conn = mysql.connector.connect(**config)
cursor = conn.cursor()

# 데이터 삽입을 위한 SQL 쿼리
insert_query = """
INSERT INTO ex_table (
    state, date, cash_buy, cash_sell, remit_send, remit_receive, 
    foreign_currency_check_sell, exchange_rate, compared_to_previous, 
    exchange_fee_rate, usd_conversion_rate
) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

# data_list의 각 데이터를 데이터베이스에 삽입
for data in data_list:
    try:
        # 데이터 매핑
        date = data[0]  # 날짜
        cash_buy = data[1]  # 현찰 사실 때
        cash_sell = data[2]  # 현찰 파실 때
        remit_send = data[3]  # 송금 보내실 때
        remit_receive = data[4]  # 송금 받으실 때
        foreign_currency_check_sell = data[5]  # 여행자수표 파실 때
        exchange_rate = data[6]  # 매매 기준율
        compared_to_previous = data[7]  # 대비
        usd_conversion_rate = data[8]  # US 달러 재정환율
        exchange_fee_rate = data[9]  # 환가료율

        # SQL 쿼리 실행
        cursor.execute(insert_query, (
            state,
            date,
            cash_buy,
            cash_sell,
            remit_send,
            remit_receive,
            foreign_currency_check_sell,
            exchange_rate,
            compared_to_previous,
            exchange_fee_rate,
            usd_conversion_rate
        ))
        # 데이터 입력
    except Exception as e:
        print(f"Error inserting data {data}: {e}")

# 변경사항 커밋
conn.commit()

# 커서와 연결 종료
cursor.close()
conn.close()

Row: ['2024-09-13', 196.87, 178.13, 189.37, 185.63, 0.0, 187.5, 0.58, 3.56333, 0.1407]
Row: ['2024-09-12', 197.48, 178.68, 189.96, 186.2, 0.0, 188.08, 0.1, 3.65197, 0.1405]
Row: ['2024-09-11', 197.37, 178.59, 189.85, 186.11, 0.0, 187.98, 0.47, 3.63121, 0.1402]
Row: ['2024-09-10', 197.87, 179.03, 190.33, 186.57, 0.0, 188.45, 0.07, 3.59227, 0.1402]
Row: ['2024-09-09', 197.94, 179.1, 190.4, 186.64, 0.0, 188.52, 0.16, 3.56773, 0.1405]
Row: ['2024-09-06', 198.11, 179.25, 190.56, 186.8, 0.0, 188.68, 0.36, 3.56773, 0.1409]
Row: ['2024-09-05', 197.73, 178.91, 190.2, 186.44, 0.0, 188.32, 0.33, 3.63394, 0.1411]
Row: ['2024-09-04', 197.38, 178.6, 189.86, 186.12, 0.0, 187.99, 0.35, 3.66742, 0.1407]
Row: ['2024-09-03', 197.75, 178.93, 190.22, 186.46, 0.0, 188.34, 0.17, 3.76152, 0.1404]
Row: ['2024-09-02', 197.57, 178.77, 190.05, 186.29, 0.0, 188.17, 0.69, 3.61758, 0.1405]
Row: ['2024-08-30', 198.3, 179.42, 190.74, 186.98, 0.0, 188.86, 0.7, 3.48818, 0.141]
Row: ['2024-08-29', 197.56, 178.76, 190.04,

Row: ['2024-04-26', 199.18, 180.22, 191.59, 187.81, 0.0, 189.7, 0.19, 6.02485, 0.1376]
Row: ['2024-04-25', 198.98, 180.04, 191.4, 187.62, 0.0, 189.51, 0.03, 6.25152, 0.1378]
Row: ['2024-04-24', 198.95, 180.01, 191.37, 187.59, 0.0, 189.48, 0.04, 6.55318, 0.1375]
Row: ['2024-04-23', 198.99, 180.05, 191.41, 187.63, 0.0, 189.52, 0.79, 7.07788, 0.1377]
Row: ['2024-04-22', 199.82, 180.8, 192.21, 188.41, 0.0, 190.31, 0.01, 6.98591, 0.1379]
Row: ['2024-04-19', 199.81, 180.79, 192.2, 188.4, 0.0, 190.3, 0.09, 6.98318, 0.138]
Row: ['2024-04-18', 199.9, 180.88, 192.29, 188.49, 0.0, 190.39, 0.56, 7.08, 0.138]
Row: ['2024-04-17', 200.49, 181.41, 192.85, 189.05, 0.0, 190.95, 0.92, 6.85682, 0.1381]
Row: ['2024-04-16', 201.46, 182.28, 193.78, 189.96, 0.0, 191.87, 0.61, 6.60258, 0.1377]
Row: ['2024-04-15', 200.82, 181.7, 193.17, 189.35, 0.0, 191.26, 0.7, 6.56015, 0.1377]
Row: ['2024-04-12', 200.08, 181.04, 192.46, 188.66, 0.0, 190.56, 1.77, 6.41727, 0.1376]
Row: ['2024-04-11', 198.22, 179.36, 190.67, 18

Row: ['2023-12-08', 192.85, 174.49, 185.5, 181.84, 0.0, 183.67, 0.15, 5.27758, 0.1391]
Row: ['2023-12-07', 192.69, 174.35, 185.35, 181.69, 0.0, 183.52, 0.15, 5.0353, 0.1396]
Row: ['2023-12-06', 192.85, 174.49, 185.5, 181.84, 0.0, 183.67, 0.16, 4.97909, 0.1394]
Row: ['2023-12-05', 192.68, 174.34, 185.34, 181.68, 0.0, 183.51, 0.15, 4.85242, 0.1394]
Row: ['2023-12-04', 192.52, 174.2, 185.19, 181.53, 0.0, 183.36, 1.12, 4.89591, 0.1399]
Row: ['2023-12-01', 191.35, 173.13, 184.06, 180.42, 0.0, 182.24, 0.22, 4.885, 0.1403]
Row: ['2023-11-30', 191.12, 172.92, 183.84, 180.2, 0.0, 182.02, 1.06, 4.895, 0.1399]
Row: ['2023-11-29', 190.0, 171.92, 182.76, 179.16, 0.0, 180.96, 0.1, 4.98727, 0.14]
Row: ['2023-11-28', 189.9, 171.82, 182.66, 179.06, 0.0, 180.86, 0.89, 4.96561, 0.1401]
Row: ['2023-11-27', 190.83, 172.67, 183.56, 179.94, 0.0, 181.75, 0.91, 4.87682, 0.1396]
Row: ['2023-11-24', 191.79, 173.53, 184.48, 180.84, 0.0, 182.66, 0.56, 5.00409, 0.1399]
Row: ['2023-11-23', 191.2, 173.0, 183.92, 180.